In [ ]:
import os
os.chdir('../')

In [ ]:
from redunet import ReduNetVector
import utils_example as ue

# Packages
To install the required packages, run 
```
conda create --name redunet --file requirements.txt
```

## Hyperparameters

We first set the hyperparameters for our data:
- `data_dim`: dimension of the data
- `dataset`: choice of dataset (any option 1-8)
- `train_noise`: scale of gaussian noise added to training data
- `test_noise`: scale of gaussian noise added to testing data
- `train_samples`: number of training samples
- `test_samples`: number of testing samples

In [ ]:
dataset = 1  # can be 1 to 8
data_dim = 3
train_noise = 0.1
test_noise = 0.1
train_samples = 100
test_samples = 100

Then for ReduNet:
- `num_layers`: number of layers
- `eta`: step size $\eta$
- `eps`: distortion $\epsilon^2$

In [ ]:
num_layers = 200  # number of redunet layers
eta = 0.5
eps = 0.1
lmbda = 200

## Data

Using `generate_3d(.)` function, we sample training and testing data from a Gaussian distribution. We also preprocess the data by normalizing them with $\ell^2$ norm. 

Our input has dimensions:

-`X_train`: shape: `(train_samples, data_dim)` <br>
-`y_train`: shape: `(train_samples,)`

In [ ]:
X_train, y_train, num_classes = ue.generate_3d(dataset, train_noise, train_samples) # train
X_test, y_test, num_classes = ue.generate_3d(dataset, test_noise, test_samples) # test

In [ ]:
X_train[:3], y_train[:3]

## Model

We then initialize our network by running `init` method, which takes in training data `X_train` and labels `y_train`. It also outputs the representations `Z_train`. While constructing, our network also computes the `total_loss`, `discrimn_loss` and `compress_loss`. Please refer to our paper for its formulation. 

In [ ]:
net = ReduNetVector(num_classes, data_dim, num_layers=num_layers, eta=eta, eps=eps)
Z_train = net.init(X_train, y_train)

## Training

In [ ]:
ue.plot_loss_mcr(net.get_loss())
ue.plot_3d(X_train, y_train, 'X_train') 
ue.plot_3d(Z_train, y_train, 'Z_train') 

## Testing

In [ ]:
Z_test = net(X_test).detach()
ue.plot_3d(X_test, y_test, 'X_test')
ue.plot_3d(Z_test, y_test, 'Z_test')

In [ ]:
!pip3 install redunet

In [1]:
from redunet import ReduNetVector

ModuleNotFoundError: No module named 'redunet'